<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# LangGraph: Evaluator–Optimizer Loop
In this tutorial, we’ll build a code generation feedback loop using LangGraph — where a generator LLM writes code and an evaluator LLM provides structured reviews. This iterative pattern is useful for refining outputs over multiple steps until they meet a defined success criterion.

The workflow consists of:

A **generator LLM** that produces or revises code based on feedback.

An **evaluator LLM** that assigns a grade (pass or fail) and gives feedback if needed.

A **LangGraph state machine** that loops the generator until the evaluator approves the result.

To make this fully observable and production-grade, we’ve instrumented the graph with Phoenix tracing. This enables you to inspect each generation and evaluation step, see what the model produced, and understand why it did (or didn’t) pass.

In [ ]:
!pip install langgraph langchain langchain_community "arize-phoenix==9.0.1" arize-phoenix-otel openinference-instrumentation-langchain

In [ ]:
!pip install langchain_openai

In [ ]:
import getpass
import os

from langgraph.graph import END, START, StateGraph

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="Evaluator-Optimizer", auto_instrument=True)

# Evaluator‑Optimizer • Code‑Writing Loop
---------------------------------------
Input  : problem_spec (natural‑language description of the function/program)

Output : refined, accepted code (Python string)


In [ ]:
from typing import Literal, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

LLMs
----
• generator_llm : writes / rewrites the code

• evaluator_llm : grades the code via structured output

In [ ]:
generator_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
evaluator_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

To enable structured, reliable feedback from the evaluator LLM, we define a Pydantic schema called Review. This schema ensures that all evaluations include both a binary grade (pass or fail) and feedback if the code needs improvement.

By binding this schema to the evaluator LLM, we guarantee consistent output formatting and make it easier to route logic in the graph. This step is essential for closing the feedback loop and driving iterative optimization.

In [ ]:
class Review(BaseModel):
    grade: Literal["pass", "fail"] = Field(description="Did the code fully solve the problem?")
    feedback: str = Field(description="If grade=='fail', give concrete, actionable feedback.")


evaluator = evaluator_llm.with_structured_output(Review)

# Langgraph Shared State
Defines the shared state for the evaluator-optimizer loop, tracking the problem description, generated code, feedback, and evaluation grade.

In [ ]:
class State(TypedDict):
    problem_spec: str
    code: str
    feedback: str
    grade: str  # pass / fail

# Node Functions: Generator & Evaluator
These nodes power the evaluator–optimizer loop. The code_generator node uses the generator LLM to produce or revise Python code based on the task and prior feedback. The code_evaluator node uses a structured evaluator LLM to simulate a code review — mentally testing the code against the spec and returning a binary grade along with constructive feedback if needed. This feedback is then looped back to the generator until the code passes.

In [ ]:
def code_generator(state: State):
    """Write or refine code based on feedback."""
    prompt = (
        "You are an expert Python developer.\n"
        "Write clear, efficient, PEP‑8 compliant code that solves the task below.\n"
        "If feedback is provided, revise the previous code accordingly.\n\n"
        f"### Task\n{state['problem_spec']}\n\n"
    )
    if state.get("feedback"):
        prompt += f"### Previous Reviewer Feedback\n{state['feedback']}\n"

    msg = generator_llm.invoke(prompt)
    return {"code": msg.content}


def code_evaluator(state: State):
    """LLM reviews the code solution."""
    review = evaluator.invoke(
        [
            SystemMessage(
                content=(
                    "You are a strict code reviewer. "
                    "Run mental tests / reasoning to decide whether the code meets the spec. "
                    "If it fails, give concise actionable feedback."
                )
            ),
            HumanMessage(
                content=(
                    f"### Problem\n{state['problem_spec']}\n\n"
                    f"### Candidate Code\n{state['code']}"
                )
            ),
        ]
    )
    return {"grade": review.grade, "feedback": review.feedback}

# Routing Logic
To support iterative refinement, we define a simple routing function called route. After the code is evaluated, this function checks the grade returned by the evaluator: if the code passes, the process ends; if it fails, control loops back to the generator for another revision. This logic ensures the LLM can continuously improve its output based on feedback until it meets the quality bar.

In [ ]:
def route(state: State):
    return "Accept" if state["grade"] == "pass" else "Revise"

# Building the LangGraph
We now define our LangGraph-based workflow. Each component — the generator and evaluator — is added as a node. Directed edges specify the flow: the graph starts with generation, then moves to evaluation. Conditional edges use the routing logic to either terminate the loop (if passed) or revisit the generator (if failed). This structure enables self-correcting behavior with every iteration traceable via Phoenix.

In [ ]:
builder = StateGraph(State)

builder.add_node("code_generator", code_generator)
builder.add_node("code_evaluator", code_evaluator)

builder.add_edge(START, "code_generator")
builder.add_edge("code_generator", "code_evaluator")
builder.add_conditional_edges(
    "code_evaluator",
    route,
    {
        "Accept": END,
        "Revise": "code_generator",
    },
)

workflow = builder.compile()

# Example Usage

In [ ]:
problem = """
Write code for a complicated website with javascript features.
"""

result_state = workflow.invoke({"problem_spec": problem})
print("===== FINAL CODE =====\n")
print(result_state["code"])

# Make sure to check our your traces in Phoenix!